![title](img/ESPN.png)

In [5]:
#pip install selenium
#pip install webdriver-manager
#pip install BeautifulSoup4
#!pip install --upgrade pandas
import pandas as pd
import numpy as np
import re
import requests
import webbrowser
import seaborn as sns
import urllib.request, json 
import unicodedata
import requests
import math
import csv
import os

from unidecode import unidecode
from IPython.core.display import display, HTML
from itertools import chain
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
#from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium import webdriver
from datetime import datetime, date, timedelta
from tqdm import tqdm

pd.set_option("display.max.columns", None)
pd.set_option("display.max.rows", None)
# driver = webdriver.Chrome(ChromeDriverManager().install())

#Function Soup
def get_page(special_match):
    driver.get(special_match)
    try:
        element = driver.find_element_by_class_name('banner-actions-container')
        element.click()
    except:
        do = "Nothing"

def strip_accents(text):
    return ''.join(c for c in unicodedata.normalize('NFD', text)
                   if unicodedata.category(c) != 'Mn')



====== WebDriver manager ======
Current google-chrome version is 110.0.5481
Get LATEST driver version for 110.0.5481
Driver [C:\Users\Valentin\.wdm\drivers\chromedriver\win32\110.0.5481.77\chromedriver.exe] found in cache


In [27]:
import datetime
URL_prefix = "https://www.espn.co.uk/football/table/_/league/"
Competitions = [
#                 "ger.1", "ger.2", #Germany
#                 "ita.1", "ita.2", #Italy
#                 "fra.1", "fra.2", #France
#                 "esp.1", "esp.2", #Spain
                "eng.1", "eng.2", #England
#                 "por.1", #"por.2", #Portugal
#                 "sco.1", #Scotland
#                 "den.1", #Denmark
#                 "aut.1", #Austria
#                 "sui.1", #Suisse
#                 "bel.1", #Belgique
#                 "ned.1", #Pays-Bas
#                 "nor.1", #Norvege
#                 "swe.1", #Suede
#                 "tur.1", #Turquie
#                 "gre.1", #Grèce
#                 "rou.1", #Roumanie
#                 "isr.1", #Israël
#                 "irl.1", #Irland
#                 "nir.1", #North Irland
#                 "mex.1", #Mexique
#                 "usa.1", #USA
#                 "arg.1", #Argentina
#                 "bre.1", #Brazil       
#                 "col.1", #Colombia
#                 "chi.1", #Chile
#                 "ecu.1", #Ecuador
#                 "par.1", #Paraguay         
#                 "jpn.1", #Japan
#                 "aus.1", #Australia              
              ]

list_teams = []
for competition in tqdm(Competitions):
    u = URL_prefix + competition
    get_page(u)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    for games in soup.findAll(class_='Table__TR Table__TR--sm Table__even'):
        for linkGame in games.findAll(class_='hide-mobile'):
            for link in linkGame.findAll(class_='AnchorLink'):
                data_item = {"Competition": competition,
                             "LinkTeam": link["href"].replace("/club/", "/team/results/").replace(re.sub(".*/", "", link["href"].replace("/club/", "/team/results/")),"")
                            }
                list_teams.append(data_item)
    for games in soup.findAll(class_='filled Table__TR Table__TR--sm Table__even'):
        for linkGame in games.findAll(class_='hide-mobile'):
            for link in linkGame.findAll(class_='AnchorLink'):
                data_item = {"Competition": competition,
                             "LinkTeam": link["href"].replace("/club/", "/team/results/").replace(re.sub(".*/", "", link["href"].replace("/club/", "/team/results/")),"")
                            }
                list_teams.append(data_item)
Teams = pd.DataFrame(list_teams)
#Teams.head()

Hist = []
for i in tqdm(range(len(Teams))):
#for i in range(2):
    Year = []
    u = "https://www.espn.co.uk" + Teams.iloc[i]["LinkTeam"] + "league/" + Teams.iloc[i]["Competition"]
    get_page(u)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    for j in soup.find_all(class_='Table__Title'):
        Year.append(j.text.split(",")[1])
    for r in soup.find_all(class_='headline headline__h1 dib'):
        team = r.text.replace(" Results","")
    i=0
    for cat in soup.find_all(class_='Table__ScrollerWrapper relative overflow-hidden'):
        for linkGame in cat.find_all(class_='Table__TR Table__TR--sm Table__even'):
            cols = linkGame.find_all(class_='Table__TD')
            for c in linkGame.find_all(class_='Table__TD'):
                if(c.find('span', class_='Table__Team score Custom_Notes_Header') is not None):
                    cols[2] = c.find('span', class_='Table__Team score Custom_Notes_Header')
            if datetime.datetime.strptime(cols[0].text + Year[i], "%a, %d %b %Y") > (datetime.datetime.strptime((datetime.datetime.now() + datetime.timedelta(days=1)).strftime("%Y-%m-%d"), "%Y-%m-%d")):
                Year[i] = str(int(Year[i]) - 1)
            data_item = {"Date": cols[0].text + Year[i], 
                         "Home": cols[1].text,
                         "Score": cols[2].text,
                         "Away": cols[3].text,
                         "Team": team
                        }
            Hist.append(data_item)
        i=i+1

100%|██████████| 32/32 [00:38<00:00,  1.19s/it]


In [24]:
import datetime
History = pd.DataFrame(Hist).drop_duplicates().drop("Team", axis=1)
History[['ScoreH', 'ScoreA']] = History['Score'].str.split(' - ', expand=True).astype(int)
History[['But']] = History['ScoreH'] + History['ScoreA']
History['Result'] = np.where(History['ScoreH'] == History['ScoreA'], 'Draw',
                             np.where(History['ScoreH'] > History['ScoreA'], 'Home', 'Away'))

History['Ds'] = History['Result'].apply(lambda x: 1 if x == 'Draw' else 0)
History['Hs'] = History['Result'].apply(lambda x: 1 if x == 'Home' else 0)
History['As'] = History['Result'].apply(lambda x: 1 if x == 'Away' else 0)
History['DA'] = History['Result'].apply(lambda x: 1 if x in ['Draw', 'Away'] else 0)
History['DH'] = History['Result'].apply(lambda x: 1 if x in ['Draw', 'Home'] else 0)
History['HA'] = History['Result'].apply(lambda x: 1 if x in ['Home', 'Away'] else 0)
History['LDEMs'] = np.where(History['ScoreH'] > 0, np.where(History['ScoreA'] > 0, 1, 0), 0)
History['LDEMPs'] = np.where(History['LDEMs'] == 1, 0, 1)

History['DALDEMs'] = np.where((History['LDEMs'] == 1) & (History['DA'] == 1), 1, 0)
History['DHLDEMs'] = np.where((History['LDEMs'] == 1) & (History['DH'] == 1), 1, 0)
History['HALDEMs'] = np.where((History['LDEMs'] == 1) & (History['HA'] == 1), 1, 0)
History['DALDEMPs'] = np.where((History['LDEMPs'] == 1) & (History['DA'] == 1), 1, 0)
History['DHLDEMPs'] = np.where((History['LDEMPs'] == 1) & (History['DH'] == 1), 1, 0)
History['HALDEMPs'] = np.where((History['LDEMPs'] == 1) & (History['HA'] == 1), 1, 0)

History["Date"] = History["Date"].apply(lambda x: datetime.datetime.strptime(x,  "%a, %d %b %Y").date())
# History = History.sort_values(by=["Home", "Date"], ascending=False)
History['DateSort'] = pd.to_datetime(History['Date'], dayfirst=True)
History = History.sort_values('DateSort', ascending=False)

#Save History until today
date_folder = os.path.join(os.getcwd() + "\HistoryV1")
os.makedirs(date_folder, exist_ok=True)
History.to_csv(os.path.join(date_folder, date.today().strftime("%Y-%m-%d") + '.csv'), index=False)

In [26]:
Games=10
#History = History[pd.to_datetime(History['Date']) > pd.Timestamp('2023-01-01')]

score_ranges = list(range(0, 7))

# Loop over the score ranges and count the number of scores greater than and less than each range
for score_range in tqdm(score_ranges):
    #History[f'H>{score_range}'] = (History['ScoreH'] > score_range).astype(int)
    #History[f'A>{score_range}'] = (History['ScoreA'] > score_range).astype(int)
    #History[f'H<{score_range}'] = (History['ScoreH'] < score_range).astype(int)
    #History[f'A<{score_range}'] = (History['ScoreA'] < score_range).astype(int)
    History[f'B>{score_range}'] = (History['But'] > score_range).astype(int)
    History[f'B<{score_range}'] = (History['But'] < score_range).astype(int)
    
    History[f'DHB>{score_range}'] = ((History['But'] > score_range) & (History['DH'] == 1)).astype(int)
    History[f'DHB<{score_range}'] = ((History['But'] < score_range) & (History['DH'] == 1)).astype(int)
    
    History[f'HAB>{score_range}'] = ((History['But'] > score_range) & (History['HA'] == 1)).astype(int)
    History[f'HAB<{score_range}'] = ((History['But'] < score_range) & (History['HA'] == 1)).astype(int)
    
    History[f'DAB>{score_range}'] = ((History['But'] > score_range) & (History['DA'] == 1)).astype(int)
    History[f'DAB<{score_range}'] = ((History['But'] < score_range) & (History['DA'] == 1)).astype(int)
    
    History['n'] = 1
    
HomeStats = History.groupby('Home')[
                              [f'B>{score_range}' for score_range in score_ranges] +
                              [f'B<{score_range}' for score_range in score_ranges] +
                              [f'DHB>{score_range}' for score_range in score_ranges] +
                              [f'DHB<{score_range}' for score_range in score_ranges] +
                              [f'HAB>{score_range}' for score_range in score_ranges] +
                              [f'HAB<{score_range}' for score_range in score_ranges] +
                              [f'DAB>{score_range}' for score_range in score_ranges] +
                              [f'DAB<{score_range}' for score_range in score_ranges] +
                              ['Ds'] +
                              ['Hs'] +
                              ['As'] +
                              ['DA'] +
                              ['DH'] +
                              ['HA'] +
                              ['LDEMs'] +
                              ['LDEMPs'] +
                              ['DALDEMs'] +
                              ['DHLDEMs'] +
                              ['HALDEMs'] +
                              ['DALDEMPs'] +
                              ['DHLDEMPs'] +
                              ['HALDEMPs'] +
                              ['n']].sum()

HomeStats['Home'] = HomeStats.index
#HomeStats.reset_index(drop=True)
#HomeStats.index.name = None
HomeStats = HomeStats[HomeStats["n"]>=6]
HomeStats = HomeStats.set_index(['Home']).add_prefix('1').reset_index()

AwayStats = History.groupby('Away')[[#f'H>{score_range}' for score_range in score_ranges] +
                              #[f'A>{score_range}' for score_range in score_ranges] +
                              #[f'H<{score_range}' for score_range in score_ranges] +
                              #[f'A<{score_range}' for score_range in score_ranges] +
                               f'B>{score_range}' for score_range in score_ranges] +
                              [f'B<{score_range}' for score_range in score_ranges] +
                              [f'DHB>{score_range}' for score_range in score_ranges] +
                              [f'DHB<{score_range}' for score_range in score_ranges] +
                              [f'HAB>{score_range}' for score_range in score_ranges] +
                              [f'HAB<{score_range}' for score_range in score_ranges] +
                              [f'DAB>{score_range}' for score_range in score_ranges] +
                              [f'DAB<{score_range}' for score_range in score_ranges] +
                              ['Ds'] +
                              ['Hs'] +
                              ['As'] +
                              ['DA'] +
                              ['DH'] +
                              ['HA'] +
                              ['LDEMs'] +
                              ['LDEMPs'] +
                              ['DALDEMs'] +
                              ['DHLDEMs'] +
                              ['HALDEMs'] +
                              ['DALDEMPs'] +
                              ['DHLDEMPs'] +
                              ['HALDEMPs'] +
                              ['n']].sum()

AwayStats['Away'] = AwayStats.index
#AwayStats.reset_index(drop=True)
#AwayStats.index.name = None
AwayStats = AwayStats[AwayStats["n"]>=6]
AwayStats = AwayStats.set_index(['Away']).add_prefix('2').reset_index()

100%|██████████| 7/7 [00:00<00:00, 199.83it/s]


In [16]:
from datetime import datetime
HistoryHA = pd.DataFrame(Hist)
HistoryHA["Date"] = HistoryHA["Date"].apply(lambda x: datetime.strptime(x,  "%a, %d %b %Y").date())
HistoryHA = HistoryHA.sort_values(by=["Team", "Date"], ascending=False)
HistoryHA[['ScoreH', 'ScoreA']] = HistoryHA['Score'].str.split(' - ', expand=True).astype(int)
HistoryHA[['But']] = HistoryHA['ScoreH'] + HistoryHA['ScoreA']
HistoryHA['Result'] = np.where(HistoryHA['ScoreH'] == HistoryHA['ScoreA'], 'D',
                             np.where(np.logical_and(HistoryHA['ScoreH'] > HistoryHA['ScoreA'], HistoryHA['Team'] == HistoryHA['Home']), 'W',
                                      np.where(np.logical_and(HistoryHA['ScoreH'] < HistoryHA['ScoreA'], HistoryHA['Team'] == HistoryHA['Away']), 'W',
                                      'L')))

team = ""
stop_flags = {
    'W': True,
    'L': True,
    'D': True,
    'WD': True,
    'LD': True,
    'WL': True,
    
    'HW': True,
    'HL': True,
    'HD': True,
    'HWD': True,
    'HLD': True,
    'HWL': True,
    
    'AW': True,
    'AL': True,
    'AD': True,
    'AWD': True,
    'ALD': True,
    'AWL': True
}
counts = {
    'W': 0,
    'L': 0,
    'D': 0,
    'WD': 0,
    'LD': 0,
    'WL': 0,
    
    'HW': 0,
    'HL': 0,
    'HD': 0,
    'HWD': 0,
    'HLD': 0,
    'HWL': 0,
    
    'AW': 0,
    'AL': 0,
    'AD': 0,
    'AWD': 0,
    'ALD': 0,
    'AWL': 0
}

for index, row in HistoryHA.iterrows():
    result_mapping = {
        "L": ("LD", "WL"),
        "D": ("WD", "LD"),
        "W": ("WD", "WL"),
    }
    double1, double2 = result_mapping.get(row['Result'])
    
    for where in ["", "H", "A"]:
        if (where == "H" and row["Team"] == row["Home"]) or (where == "A" and row["Team"] == row["Away"]) or (where == ""):
            allRes = [row['Result'], double1, double2]
            if team != row['Team']:
                team = row['Team']
                stop_flags = {key: True for key in stop_flags}
                counts = {key: 0 for key in counts}

            for result in ['W', 'L', 'D', 'WD', 'LD', 'WL']:
                if stop_flags[where+result]:
                    if (result in allRes):
                        counts[where+result] += 1
                    else:
                        HistoryHA.at[index, where+result] = counts[where+result]
                        stop_flags[where+result] = False
HistoryHA.fillna(0, inplace=True)
# HistoryHA[HistoryHA["Team"] == "Borussia Dortmund"].head(15)
# HistoryHA[HistoryHA["Team"] == "Fortuna Düsseldorf"].head(20)

Streak = HistoryHA.groupby('Team')[['W','L','D','WD','LD','WL',
                                        'HW','HL','HD','HWD','HLD','HWL',
                                        'AW','AL','AD','AWD','ALD','AWL']].max()
Streak['max'] = Streak.iloc[:, 0:6].max(axis=1)
Streak['maxH'] = Streak.iloc[:, 6:12].max(axis=1)
Streak['maxA'] = Streak.iloc[:, 12:18].max(axis=1)

mini = 10
Streak = Streak.sort_values(['max', 'maxH'], ascending=False)
# Streak = Streak[(Streak["maxH"] >= mini) | (Streak["maxA"] >= mini)]#.head(20)
Streak['Team'] = Streak.index
Streak = Streak.set_index(['Team']).reset_index()
Streak.style.background_gradient(cmap="viridis").set_precision(0)